# An AS/AD-model for a closed economy in the short run

**This project analyzes the effects of shocks to a simple AS/AD-model for a closed economy in the short run. We use the model to analyze supply- and demand shocks as well as minimizing a social loss function.**

Imports and set magics:

In [12]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider
import scipy.optimize as opt

plt.rcParams.update({"axes.grid": True, "grid.color": "darkblue", "grid.alpha": "0.3", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 12})

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

import modelproject
from modelproject import EconomicModel


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

The model used in this project is a version of a closed economy AS/AD-model as outlined in "Introducing Advanced Macroeconomics - Growth and Business Cycles" by Peter Birch Sørensen & Hans Jørgen Whitta-Jacobsen.  

We use the following equations to describe our model: 

**1: IS-curve** - Market equilibrium with government spending and taxes.

$$
y_t-\bar{y} = \alpha_1 (g_t-\bar{g}) - \alpha_2(r_t-\bar{r})-\alpha_{3}(\tau_{t}-\overline{\tau})
$$

**2: Fischer Equation** - In ex-ante (modeled by expected inflation).

$$
r_t = i_t-\pi^{e}_{t+1}
$$

**3: Monetary policy rule** - The Taylor-Rule with stabilization wrt. both inflation and output

$$
i_t = \bar{r} + \pi^{e}_{t+1} + h(\pi_t - \pi^*) + b(y_t- \bar{y})
$$

**4: The SRAS curve** - This is derived from the expectations-augmented Phillips-curve. For given inflation expectations, there is a positive correlation between inflation and outputgap. 

$$
\pi_t = \pi_t^e + \gamma (y_t-\bar{y}) + s_t
$$


**5: Inflation expectations** - This equation shows that the agents have static inflation expectations, since they expect inflation in period t to be equal to observed inflation in the previous period. 

$$
\pi_t^e = \pi_{t-1}
$$

From the above equations, we can define the AD-curve and the AS-curve: 


The AD-curve combines equations 1, 2 and 3. Inserting 3 in 2:

$$
r_t = \bar{r} + \pi^{e}_{t+1} + h(\pi_t - \pi^*) + b(y_t- \bar{y})-\pi^{e}_{t+1}
$$
$$
r_t = \bar{r} + h(\pi_t - \pi^*) + b(y_t- \bar{y})
$$
Inserting in eq. 1:
$$
y_t-\bar{y} = \alpha_1 (g_t-\bar{g}) - \alpha_2(\bar{r} + h(\pi_t - \pi^*) + b(y_t- \bar{y})-\bar{r})-\alpha_{3}(\tau_{t}-\overline{\tau})
$$
$$
y_t-\bar{y} = \alpha_1 (g_t-\bar{g}) - \alpha_2(h(\pi_t - \pi^*) + b(y_t- \bar{y}))-\alpha_{3}(\tau_{t}-\overline{\tau})
$$
Isolating $y_t-\bar{y}$:
$$
y_t-\bar{y} = z-\alpha(\pi_t - \pi^*)
$$
where
$$
z = \frac{\alpha_1}{1+\alpha_2b}(g_t-\bar{g})-\frac{\alpha_3}{1+\alpha_2b}(\tau_t-\bar{\tau}), \alpha = \frac{\alpha_2 h}{1+\alpha_2b}
$$

The AS-curve is found by inserting equation 5 in 4:

$$
\pi_t = \pi_{t-1} + \gamma(y_t-\bar{y}) + s_t $$

## Analytical solution

We firstly define our model in python using a class in our py-file. We use the model to make plots with interactive sliders for the demand shock, z, and the supply shock, s. 



In [14]:
def interactive_gap(periods, demand_shock, supply_shock, demand_duration, supply_duration):
    model = EconomicModel(periods=periods, demand_shock=demand_shock, supply_shock=supply_shock, 
                          demand_duration=demand_duration, supply_duration=supply_duration)
    model.calculate_model()
    model.display_results()

# Create sliders for interactive input
periods_slider = IntSlider(min=10, max=100, step=1, value=20, description='Total Periods')
demand_shock_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.01, description='Demand Shock')
supply_shock_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0, description='Supply Shock')
demand_duration_slider = IntSlider(min=0, max=50, step=1, value=1, description='Duration of Demand Shock')
supply_duration_slider = IntSlider(min=0, max=50, step=1, value=0, description='Duration of Supply Shock')

# Display the interactive plot
interactive_plot = interactive(interactive_gap, periods=periods_slider, demand_shock=demand_shock_slider, 
                               supply_shock=supply_shock_slider, demand_duration=demand_duration_slider, 
                               supply_duration=supply_duration_slider)
display(interactive_plot)


interactive(children=(IntSlider(value=20, description='Total Periods', min=10), FloatSlider(value=0.01, descri…

**Temporary demand shock:**
In Figure 1, the initial values show a demand shock, z, of value 0.01, with a duration of 1 period (and no supply shock). This shock could arise from increased government spending, reduced taxes, or adjustments in monetary policy parameters such as the sensitivity of nominal interest rates to output gap changes. Initially, both the output gap and inflation rise in response to this positive shock. In the subsequent period, although the demand shock dissipates, static inflation expectations persist among economic agents, leading them to anticipate continued higher inflation. However, actual inflation turns out to be lower than expected, resulting in a negative output gap. As agents gradually revise their expectations to match observed inflation, inflation converges back to its original level post-shock.

**Temporary supply shock:**
If we instead change the sliders to a negative value of $s=-0.01$ and a duration of s of 1 (and no demand shock), this corresponds to a positive supply shock (e.g., more production at lower costs). We find that the output gap increases and after the first period converges back to zero, while the inflation gap becomes negative and gradually converges back to zero. We will elaborate on the effects in the AS-AD-plot below. 

In [15]:
def interactive_model(periods, demand_shock, supply_shock, gamma, demand_duration, supply_duration):
    model = EconomicModel(periods=periods, gamma=gamma, demand_shock=demand_shock, supply_shock=supply_shock, 
                          demand_duration=demand_duration, supply_duration=supply_duration)
    model.calculate_model()
    model.display_ad_as()

# Create sliders for interactive input
periods_slider = IntSlider(min=1, max=50, step=1, value=20, description='Total Periods')
demand_shock_slider = FloatSlider(min=-0.1, max=0.1, step=0.01, value=0.01, description='Demand Shock')
supply_shock_slider = FloatSlider(min=-0.1, max=0.1, step=0.01, value=0, description='Supply Shock')
gamma_slider = FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Gamma')
demand_duration_slider = IntSlider(min=0, max=50, step=1, value=1, description='Duration of Demand Shock')
supply_duration_slider = IntSlider(min=0, max=50, step=1, value=0, description='Duration of Supply Shock')

# Display interactive widgets
interactive_plot = interactive(interactive_model, periods=periods_slider, demand_shock=demand_shock_slider, 
                               supply_shock=supply_shock_slider, gamma=gamma_slider, 
                               demand_duration=demand_duration_slider, supply_duration=supply_duration_slider)
display(interactive_plot)


interactive(children=(IntSlider(value=20, description='Total Periods', max=50, min=1), FloatSlider(value=0.01,…

Figure 2 shows the AS-AD-model and includes sliders for the shocks, as in Figure 1. 

**Temporary demand shock:**
The initial values show a demand shock, z, of value 0.01, with a duration of 1 period (and no supply shock). We see that in period 1, the demand shock raises output and inflation above long-term levels (economic boom). Increased demand boosts production and employment, leading to higher costs and inflation. The central bank responds with a tightening policy more than one-for-one with inflation, raising real interest rates and dampening demand. In period 2, the AD-curve shifts back to the initial AD-curve, as the demand shock fades. Higher inflation expectations from period 1 shift AS left/up in period 2, anticipating inflation. This drives wage demands, accelerates prices, and sustains elevated inflation. The central bank continues tightening to stabilize inflation. In the subsequent periods, inflation expectations adjust to period the former periods’ observed inflation, shifting AS downward. Over time, output returns to its long-term level and inflation stabilizes. Nominal wage rigidities and slow expectation adjustments influence this process.

**Temporary supply shock:**
If we instead change the sliders to a negative value of $s=-0.01$ and a duration of s of 1 (and no demand shock), this corresponds to a positive supply shock, like in the analysis for Figure 1 above. In the first period, a temporary positive supply shock ($s < 0$) shifts AS down by the size of the shock s, creating a new short-term equilibrium. Inflation falls to $\pi_1 < \pi^*$, but not entirely with the shock’s size, as the central bank responds by easing monetary policy (lowering interest rates), boosting output ($y_1>\bar{y}$). In the second period where $s=0$, AS shifts up. However, static expectations prompt the agents to adjust inflation expectations, keeping the AS-curve from returning directly. In all subsequent periods, inflation expectations adjust to observed inflation in the former period, pushing AS gradually up. Successive expectation adjustments lead output towards $\bar{y}$ and inflation towards $\pi^*$. Persistence in convergence is due to nominal wage rigidities and slow adjustment of expectations.

In [16]:
def interactive_permanent_shocks(total_periods, demand_shock, supply_shock, gamma_value):
    demand_duration = supply_duration = total_periods  # Set both durations equal to total_periods for permanent shocks
    model = EconomicModel(periods=total_periods, gamma=gamma_value, demand_shock=demand_shock, supply_shock=supply_shock, 
                          demand_duration=demand_duration, supply_duration=supply_duration)
    model.calculate_model()
    model.display_ad_as()

# Create sliders for interactive input
total_periods_slider = IntSlider(min=1, max=50, step=1, value=20, description='Total Periods')
demand_shock_slider = FloatSlider(min=-0.1, max=0.1, step=0.001, value=0.01, description='Demand Shock', readout_format='.3f')
supply_shock_slider = FloatSlider(min=-0.01, max=0.01, step=0.001, value=0, description='Supply Shock', readout_format='.3f')
gamma_value_slider = FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Gamma')

# Display the interactive plot
interactive_plot = interactive(interactive_permanent_shocks, total_periods=total_periods_slider, demand_shock=demand_shock_slider, 
                               supply_shock=supply_shock_slider, gamma_value=gamma_value_slider)
display(interactive_plot)


interactive(children=(IntSlider(value=20, description='Total Periods', max=50, min=1), FloatSlider(value=0.01,…

In Figure 3, the durations of the shocks are set equal to T (total periods), so changing the slider for the shocks shows the effects of permanent shocks in the AS-AD-model. 

**Permanent demand shocks:**
We find that demand shocks do not affect the economy in the long run. In the long run, a demand shock in an AS-AD economy leads to the economy returning to its natural output level, $\bar{y}$. Initially higher output and inflation caused by increased demand gradually stabilize as prices, wages, and inflation expectations adjust back to their long-term equilibrium levels. Central banks typically adjust monetary policy to ensure inflation returns to its target, contributing to the economy's eventual return to equilibrium.

**Permanent supply shocks:**
Supply shocks, however, affect both the AS and AD-curves in the long run. When there is a positive supply shock, such as technological advancements or increased productivity, the AS curve shifts to the right, reflecting higher potential output levels and lower costs for producers. This shift leads to:
* Gradual decreases in prices as goods become more abundant and cheaper to produce.
* Initially, an increase in real incomes and purchasing power due to lower prices.
* Over time, the AD curve shifts rightward as consumers and businesses respond to lower prices by increasing their spending and investment.
Conversely, a negative supply shock, like higher production costs or reduced productivity, shifts the AS curve to the left:
* Output decreases as it becomes more costly to produce goods and services.
* Prices rise as supply becomes constrained, reducing consumer purchasing power.
* Initially, the AD curve may shift leftward as higher prices reduce consumer spending and investment.


## Social loss function 

In this section, we look at the social loss of shocks to inflation and output. Social loss is minimized when the following function is minimized:
$$ L_t = (\pi_t - \pi^*)^2 + \alpha (y_t - \bar{y})^2 $$


In [17]:
def interactive_social_loss(total_periods, demand_shock, supply_shock, demand_shock_duration, supply_shock_duration):
    model = EconomicModel(periods=total_periods, demand_shock=demand_shock, supply_shock=supply_shock, 
                          demand_duration=demand_shock_duration, supply_duration=supply_shock_duration)
    model.calculate_model()
    model.display_social_loss()

# Create sliders for interactive input
total_periods_slider = IntSlider(min=10, max=100, step=1, value=20, description='Total Periods')
demand_shock_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.01, description='Demand Shock')
supply_shock_slider = FloatSlider(min=-0.2, max=0.2, step=0.01, value=0, description='Supply Shock')
demand_shock_duration_slider = IntSlider(min=0, max=50, step=1, value=1, description='Duration of Demand Shock')
supply_shock_duration_slider = IntSlider(min=0, max=50, step=1, value=0, description='Duration of Supply Shock')

# Display the interactive plot
interactive_plot = interactive(interactive_social_loss, total_periods=total_periods_slider, demand_shock=demand_shock_slider, 
                               supply_shock=supply_shock_slider, demand_shock_duration=demand_shock_duration_slider, 
                               supply_shock_duration=supply_shock_duration_slider)
display(interactive_plot)


interactive(children=(IntSlider(value=20, description='Total Periods', min=10), FloatSlider(value=0.01, descri…

Figure 4 illustrates the social loss function resulting from the different shocks from adjusting the sliders. Initially, both shocks lead to increased social costs, peaking during the shock periods. Over time, social loss gradually diminishes and approaches zero.
This trend highlights that while initial shocks disrupt economic stability, the economy adapts and mitigates these impacts over the longer term. Ultimately, the convergence to zero reflects the economy's ability to return to equilibrium despite temporary disturbances.

## Numerical solution

We now compute the numerical solution to the model. We create an instance of EconomicModel with 100 periods, a demand shock of 0.05, and a supply shock of 0.02. We find the optimal value of alpha that minimizes the output gap after the shock. In Hans Jørgen Whitta-Jacobsen and Peter Birch Sørensen’s book, "Introduction to Advanced Macroeconomics," the optimal parameter value of $\gamma = 0.075$, which we utilize for the numerical solution.

In [23]:
# Optimizing the model
model = EconomicModel(periods=100, demand_shock=0.05, supply_shock=0.02, gamma=0.075)
optimal_alpha, optimal_output_gap = model.optimize_output_gap()

# Printing the results
print(f"Optimal Alpha: {optimal_alpha}")
print(f"Optimal Output Gap after Shock: {optimal_output_gap}")


Optimal Alpha: 2.4999999948197824
Optimal Output Gap after Shock: 8.117528284510844e-11


We find that the optimal alpha is 2.5, which gives an outputgap after the shock of approx. zero. This alpha is higher than the empirical value of 0.7 from Whitta-Jacobsen and Birch Sørensen. In the numerical solution, we aim to minimize outputgap after the given shocks, why it is not surprising that we find a high alpha-value. A high alpha-value suggest an extremely high sensitivity of output to changes in inflation and supply shocks. Such a high alpha value implies that even small changes in inflation or supply shocks lead to very large adjustments in output from the central bank. This is due to a high value of h, meaning that the central bank adjusts the interest rate more when there are changes in inflation. 

## Conclusion

In conclusion, this project has examined the impact of shocks on a basic AS/AD model within a closed economy in the short term. Through our analysis, we investigated the repercussions of variations in shocks and a social loss function, and found the optimal alpha to minimize outputgap after shocks.